In [1]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.preprocessing import FunctionTransformer

In [2]:
df = pd.read_csv("../data/preprocessed/final_etf_data/weekends_aggregated_embedding/XLY_v4.csv")
df = sign_next_day(df)
df = drop_sign_and_return(df)


In [3]:
df.head(10)

,Date,Price,avg_positive_XLY,avg_neutral_XLY,avg_negative_XLY,n_XLY,sent_index_XLY,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,97.193489,0.178093,0.597175,0.224732,6.0,-0.046640,0.016718,-0.021112,-0.005766,...,0.006048,-0.031161,-0.027777,0.023784,0.029814,-0.083138,0.007425,0.056890,1.0,-1.0
1,2018-03-21,97.119080,0.168511,0.368897,0.462592,7.0,-0.294081,-0.005555,-0.007539,-0.003753,...,0.008299,-0.013918,-0.022706,0.025375,0.054496,-0.049718,0.035943,0.013483,1.0,-1.0
2,2018-03-22,95.054108,0.291419,0.317227,0.391354,12.0,-0.099935,-0.004012,-0.004603,-0.003922,...,-0.046244,0.001894,-0.031362,-0.025443,-0.012649,-0.075833,-0.011973,0.040940,1.0,-1.0
3,2018-03-23,93.184464,0.464723,0.217883,0.317395,6.0,0.147328,0.009553,0.003357,0.030665,...,-0.038668,-0.004425,-0.017456,-0.033640,0.051639,-0.055070,-0.028548,0.008049,1.0,1.0
4,2018-03-26,95.937767,0.156581,0.449208,0.394211,12.0,-0.237630,0.013959,0.019697,0.008585,...,-0.009761,-0.010656,-0.035896,0.007026,0.016796,-0.088669,0.019996,0.024209,1.0,-1.0
5,2018-03-27,94.058807,0.145931,0.566414,0.287655,13.0,-0.141724,0.020231,0.008901,-0.009654,...,-0.003480,-0.024857,-0.005074,0.025271,0.024286,-0.041200,0.019293,0.014414,1.0,-1.0
6,2018-03-28,92.914688,0.110499,0.288523,0.600978,13.0,-0.490478,0.025747,0.000322,0.009955,...,-0.032413,-0.017052,-0.031439,-0.017196,0.006576,-0.044247,-0.004078,0.027659,1.0,1.0
7,2018-03-29,94.216949,0.192724,0.536793,0.270483,16.0,-0.077759,0.008790,0.001489,0.025637,...,-0.011518,-0.013490,-0.020716,0.010989,0.023757,-0.079593,0.009825,0.058976,1.0,-1.0
8,2018-04-02,91.556648,0.071707,0.381009,0.547284,22.0,-0.475577,0.018050,-0.004851,0.003763,...,-0.012431,-0.011981,-0.029470,-0.008287,0.027209,-0.081962,0.024847,0.043643,1.0,1.0
9,2018-04-03,92.719345,0.235805,0.338093,0.426103,14.0,-0.190298,0.035091,-0.033420,-0.013055,...,-0.011171,-0.017822,-0.015672,0.014467,0.025899,-0.107842,-0.008392,0.042751,1.0,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Columns: 393 entries, Date to Sign_next_day
dtypes: float64(392), object(1)
memory usage: 1.8+ MB


In [5]:
df.describe()

,Price,avg_positive_XLY,avg_neutral_XLY,avg_negative_XLY,n_XLY,sent_index_XLY,emb_0,emb_1,emb_2,emb_3,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
count,587.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,...,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.0,586.000000
mean,108.061448,0.208476,0.392737,0.398787,8.206485,-0.190312,0.003537,-0.006522,0.019923,0.010993,...,-0.017585,-0.009553,-0.022713,-0.001007,0.016396,-0.075259,0.002778,0.032379,1.0,0.139932
std,9.101649,0.123483,0.151819,0.166552,3.732055,0.250851,0.020359,0.020464,0.017968,0.019590,...,0.018727,0.018964,0.021152,0.020446,0.019024,0.023240,0.021950,0.018184,0.0,0.991007
min,83.233559,0.009690,0.018847,0.011134,1.000000,-0.959376,-0.058688,-0.092197,-0.046444,-0.117748,...,-0.069782,-0.146051,-0.101033,-0.100607,-0.054869,-0.145543,-0.063598,-0.037732,1.0,-1.000000
25%,101.366055,0.120003,0.290881,0.284752,6.000000,-0.357634,-0.009290,-0.019493,0.008393,0.000883,...,-0.029888,-0.020709,-0.034990,-0.013819,0.004080,-0.091332,-0.009793,0.020716,1.0,-1.000000
50%,108.347176,0.195542,0.391714,0.390323,8.000000,-0.177627,0.004245,-0.006336,0.019221,0.011312,...,-0.018160,-0.010212,-0.022963,-0.001079,0.016296,-0.075523,0.002834,0.032154,1.0,1.000000
75%,115.048347,0.270549,0.491275,0.504906,10.000000,-0.032091,0.016374,0.006184,0.030972,0.022565,...,-0.006770,0.002154,-0.009462,0.012821,0.028626,-0.060400,0.015213,0.044578,1.0,1.000000
max,129.252106,0.650597,0.859774,0.970264,24.000000,0.527286,0.095932,0.056248,0.084398,0.076313,...,0.072556,0.061982,0.118773,0.066832,0.068869,0.031759,0.122146,0.094439,1.0,1.000000


In [6]:
df.isna().sum()

Date                0
Price               0
avg_positive_XLY    1
avg_neutral_XLY     1
avg_negative_XLY    1
                   ..
emb_381             1
emb_382             1
emb_383             1
is_trading_day      1
Sign_next_day       1
Length: 393, dtype: int64

In [7]:
df.nunique()

Date                587
Price               568
avg_positive_XLY    586
avg_neutral_XLY     586
avg_negative_XLY    586
                   ... 
emb_381             586
emb_382             586
emb_383             586
is_trading_day        1
Sign_next_day         2
Length: 393, dtype: int64

### defining target - I want to assign:

class 1 to sign_next_day ==1

and class 0 to sign_next_day ∈ {0, -1}

In [8]:
df = df.dropna(subset=["Sign_next_day"])
df["Sign_next_day"] = (df["Sign_next_day"] == 1).astype(int)
df.head()

,Date,Price,avg_positive_XLY,avg_neutral_XLY,avg_negative_XLY,n_XLY,sent_index_XLY,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,97.193489,0.178093,0.597175,0.224732,6.0,-0.046640,0.016718,-0.021112,-0.005766,...,0.006048,-0.031161,-0.027777,0.023784,0.029814,-0.083138,0.007425,0.056890,1.0,0
1,2018-03-21,97.119080,0.168511,0.368897,0.462592,7.0,-0.294081,-0.005555,-0.007539,-0.003753,...,0.008299,-0.013918,-0.022706,0.025375,0.054496,-0.049718,0.035943,0.013483,1.0,0
2,2018-03-22,95.054108,0.291419,0.317227,0.391354,12.0,-0.099935,-0.004012,-0.004603,-0.003922,...,-0.046244,0.001894,-0.031362,-0.025443,-0.012649,-0.075833,-0.011973,0.040940,1.0,0
3,2018-03-23,93.184464,0.464723,0.217883,0.317395,6.0,0.147328,0.009553,0.003357,0.030665,...,-0.038668,-0.004425,-0.017456,-0.033640,0.051639,-0.055070,-0.028548,0.008049,1.0,1
4,2018-03-26,95.937767,0.156581,0.449208,0.394211,12.0,-0.237630,0.013959,0.019697,0.008585,...,-0.009761,-0.010656,-0.035896,0.007026,0.016796,-0.088669,0.019996,0.024209,1.0,0


In [9]:
df.shape

(586, 393)

In [10]:
df.isna().sum()

Date                0
Price               0
avg_positive_XLY    1
avg_neutral_XLY     1
avg_negative_XLY    1
                   ..
emb_381             1
emb_382             1
emb_383             1
is_trading_day      1
Sign_next_day       0
Length: 393, dtype: int64

In [11]:
df.head()

,Date,Price,avg_positive_XLY,avg_neutral_XLY,avg_negative_XLY,n_XLY,sent_index_XLY,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,97.193489,0.178093,0.597175,0.224732,6.0,-0.046640,0.016718,-0.021112,-0.005766,...,0.006048,-0.031161,-0.027777,0.023784,0.029814,-0.083138,0.007425,0.056890,1.0,0
1,2018-03-21,97.119080,0.168511,0.368897,0.462592,7.0,-0.294081,-0.005555,-0.007539,-0.003753,...,0.008299,-0.013918,-0.022706,0.025375,0.054496,-0.049718,0.035943,0.013483,1.0,0
2,2018-03-22,95.054108,0.291419,0.317227,0.391354,12.0,-0.099935,-0.004012,-0.004603,-0.003922,...,-0.046244,0.001894,-0.031362,-0.025443,-0.012649,-0.075833,-0.011973,0.040940,1.0,0
3,2018-03-23,93.184464,0.464723,0.217883,0.317395,6.0,0.147328,0.009553,0.003357,0.030665,...,-0.038668,-0.004425,-0.017456,-0.033640,0.051639,-0.055070,-0.028548,0.008049,1.0,1
4,2018-03-26,95.937767,0.156581,0.449208,0.394211,12.0,-0.237630,0.013959,0.019697,0.008585,...,-0.009761,-0.010656,-0.035896,0.007026,0.016796,-0.088669,0.019996,0.024209,1.0,0


In [12]:
df_catboost = df.copy()
df_catboost.head()

,Date,Price,avg_positive_XLY,avg_neutral_XLY,avg_negative_XLY,n_XLY,sent_index_XLY,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,97.193489,0.178093,0.597175,0.224732,6.0,-0.046640,0.016718,-0.021112,-0.005766,...,0.006048,-0.031161,-0.027777,0.023784,0.029814,-0.083138,0.007425,0.056890,1.0,0
1,2018-03-21,97.119080,0.168511,0.368897,0.462592,7.0,-0.294081,-0.005555,-0.007539,-0.003753,...,0.008299,-0.013918,-0.022706,0.025375,0.054496,-0.049718,0.035943,0.013483,1.0,0
2,2018-03-22,95.054108,0.291419,0.317227,0.391354,12.0,-0.099935,-0.004012,-0.004603,-0.003922,...,-0.046244,0.001894,-0.031362,-0.025443,-0.012649,-0.075833,-0.011973,0.040940,1.0,0
3,2018-03-23,93.184464,0.464723,0.217883,0.317395,6.0,0.147328,0.009553,0.003357,0.030665,...,-0.038668,-0.004425,-0.017456,-0.033640,0.051639,-0.055070,-0.028548,0.008049,1.0,1
4,2018-03-26,95.937767,0.156581,0.449208,0.394211,12.0,-0.237630,0.013959,0.019697,0.008585,...,-0.009761,-0.010656,-0.035896,0.007026,0.016796,-0.088669,0.019996,0.024209,1.0,0


In [13]:
df_catboost["Date"] = pd.to_datetime(df_catboost["Date"]).dt.normalize()
df_catboost = df_catboost.sort_values("Date").reset_index(drop=True)

df_catboost["no_news"] = df_catboost["n_XLY"].isna().astype(int) # a day where there were no sector specific news
df_catboost.head(10)


,Date,Price,avg_positive_XLY,avg_neutral_XLY,avg_negative_XLY,n_XLY,sent_index_XLY,emb_0,emb_1,emb_2,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,2018-03-20,97.193489,0.178093,0.597175,0.224732,6.0,-0.046640,0.016718,-0.021112,-0.005766,...,-0.031161,-0.027777,0.023784,0.029814,-0.083138,0.007425,0.056890,1.0,0,0
1,2018-03-21,97.119080,0.168511,0.368897,0.462592,7.0,-0.294081,-0.005555,-0.007539,-0.003753,...,-0.013918,-0.022706,0.025375,0.054496,-0.049718,0.035943,0.013483,1.0,0,0
2,2018-03-22,95.054108,0.291419,0.317227,0.391354,12.0,-0.099935,-0.004012,-0.004603,-0.003922,...,0.001894,-0.031362,-0.025443,-0.012649,-0.075833,-0.011973,0.040940,1.0,0,0
3,2018-03-23,93.184464,0.464723,0.217883,0.317395,6.0,0.147328,0.009553,0.003357,0.030665,...,-0.004425,-0.017456,-0.033640,0.051639,-0.055070,-0.028548,0.008049,1.0,1,0
4,2018-03-26,95.937767,0.156581,0.449208,0.394211,12.0,-0.237630,0.013959,0.019697,0.008585,...,-0.010656,-0.035896,0.007026,0.016796,-0.088669,0.019996,0.024209,1.0,0,0
5,2018-03-27,94.058807,0.145931,0.566414,0.287655,13.0,-0.141724,0.020231,0.008901,-0.009654,...,-0.024857,-0.005074,0.025271,0.024286,-0.041200,0.019293,0.014414,1.0,0,0
6,2018-03-28,92.914688,0.110499,0.288523,0.600978,13.0,-0.490478,0.025747,0.000322,0.009955,...,-0.017052,-0.031439,-0.017196,0.006576,-0.044247,-0.004078,0.027659,1.0,1,0
7,2018-03-29,94.216949,0.192724,0.536793,0.270483,16.0,-0.077759,0.008790,0.001489,0.025637,...,-0.013490,-0.020716,0.010989,0.023757,-0.079593,0.009825,0.058976,1.0,0,0
8,2018-04-02,91.556648,0.071707,0.381009,0.547284,22.0,-0.475577,0.018050,-0.004851,0.003763,...,-0.011981,-0.029470,-0.008287,0.027209,-0.081962,0.024847,0.043643,1.0,1,0
9,2018-04-03,92.719345,0.235805,0.338093,0.426103,14.0,-0.190298,0.035091,-0.033420,-0.013055,...,-0.017822,-0.015672,0.014467,0.025899,-0.107842,-0.008392,0.042751,1.0,1,0


In [14]:
df_catboost = df_catboost.drop(columns=["Date"])
df_catboost.head(1)

,Price,avg_positive_XLY,avg_neutral_XLY,avg_negative_XLY,n_XLY,sent_index_XLY,emb_0,emb_1,emb_2,emb_3,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,97.193489,0.178093,0.597175,0.224732,6.0,-0.04664,0.016718,-0.021112,-0.005766,-0.017972,...,-0.031161,-0.027777,0.023784,0.029814,-0.083138,0.007425,0.05689,1.0,0,0


In [15]:
original_features = [
    'Price',
    'avg_positive_XLY',
    'avg_neutral_XLY', 
    'avg_negative_XLY',
    'n_XLY',
    'sent_index_XLY',
    'is_trading_day'
]


embedding_features = [f'emb_{i}' for i in range(384)]

target = 'Sign_next_day'
df_clean = df_catboost.dropna(subset=[target, 'is_trading_day']).copy()


for emb_col in embedding_features:
    df_clean[emb_col] = df_clean[emb_col].fillna(0)


df_clean['avg_positive_XLY'] = df_clean['avg_positive_XLY'].fillna(0)
df_clean['avg_neutral_XLY'] = df_clean['avg_neutral_XLY'].fillna(1)
df_clean['avg_negative_XLY'] = df_clean['avg_negative_XLY'].fillna(0)
df_clean['n_XLY'] = df_clean['n_XLY'].fillna(0)
df_clean['sent_index_XLY'] = df_clean['sent_index_XLY'].fillna(0)

print(f"Clean dataset size: {len(df_clean)} rows")


Clean dataset size: 585 rows


In [16]:
X = df_clean[original_features + embedding_features]
y = df_clean[target]

#y = (y == 1).astype(int)

print(f"Target distribution: {y.value_counts().to_dict()}")

Target distribution: {1: 333, 0: 252}


In [17]:
embedding_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=12))  # 95% variance
])


original_pipeline = Pipeline([
    ('passthrough', FunctionTransformer())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('original', original_pipeline, original_features),
        ('embeddings', embedding_pipeline, embedding_features)
    ],
    remainder='drop'
)

In [18]:
catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    train_dir="cb_tmp",
    allow_writing_files=False
)

full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', catboost)
])


param_distributions = {
    'preprocessor__embeddings__pca__n_components': [0.90, 0.95, 0.98, 50, 75, 100],
    'classifier__depth': randint(3, 7),
    'classifier__learning_rate': uniform(0.01, 0.19),
    'classifier__iterations': randint(50, 500),
    'classifier__l2_leaf_reg': uniform(1, 14),
    'classifier__border_count': randint(32, 128),
    'classifier__random_strength': uniform(0.5, 5),
    'classifier__min_data_in_leaf': randint(5, 30),
    'classifier__rsm': uniform(0.6, 0.4),
    'classifier__boosting_type': ['Ordered', 'Plain'],
    'classifier__bootstrap_type': ['Bernoulli', 'MVS'],
    'classifier__subsample': uniform(0.6, 0.4),
}

random_search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_distributions,
    n_iter=40,
    cv=5,
    scoring='roc_auc',
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)


random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print(f"Best parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")



Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.5831
Best parameters:
  classifier__boosting_type: Ordered
  classifier__bootstrap_type: MVS
  classifier__border_count: 105
  classifier__depth: 6
  classifier__iterations: 347
  classifier__l2_leaf_reg: 10.250296467875057
  classifier__learning_rate: 0.1265526717623953
  classifier__min_data_in_leaf: 25
  classifier__random_strength: 2.369043087644187
  classifier__rsm: 0.8819847121033066
  classifier__subsample: 0.7957232605766812
  preprocessor__embeddings__pca__n_components: 75


In [ ]:
best_preprocessor = random_search.best_estimator_.named_steps['preprocessor']
pca = best_preprocessor.named_transformers_['embeddings'].named_steps['pca']

print(f"Number of prncipals components kept: {pca.n_components_}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Original embedding dimensions (384) were reduced to {pca.n_components_}")


Number of prncipals components kept: 75
Explained variance ratio: 0.7810
Original embedding dimensions (384) were reduced to 75


In [3]:
import pandas as pd

df = pd.read_csv('final_data_with_embeddings_without_aggregations.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'final_data_with_embeddings_without_aggregations.csv'

In [2]:
import os
os.getcwd()

'C:\\Users\\leszc\\AppData\\Local\\Programs\\Microsoft VS Code'